<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/TIC/Projeto_C1_avaliacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/logo_mackenzie.png'>

# **Avaliação Projeto C1**

Relembrando, como descrito no plano da disciplina, o Projeto C1 consiste em uma atividade individual onde os alunos terão a opção de participar de uma competição com o intuito de resolver um problema de classificação. Os três alunos cujos modelos levarem aos maiores desempenhos do classificador, medido em termos da acurácia, terão a nota C1 igual a 1. Em caso de empate, será utilizada uma métrica de equidade apresentada no dia da avaliação do projeto. Os alunos vencedores também explicarão para o restante da turma as estratégias utilizadas para a construção da inteligência computacional.

Para esta atividade, usamos a base de dados [COMPAS Recidivism Risk](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing), da ProPublica. Essa base de dados ficou conhecida por apresentar vieses raciais.

# Avaliando o modelo treinado

Para avaliar o modelo, use o conjunto de dados disponível no link abaixo:

https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Datasets/data_compas_projC1_test.csv





In [ ]:
# Lendo os dados de teste

dados_teste = pd.read_csv('https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Datasets/data_compas_projC1_test.csv')
dados_test.head()

Primeiro, separe os dados de entrada (`X_test`) e os dados de saída (`y_test`). Depois, ajuste os dados de teste para poder usá-lo no modelo treinado:

- Caso tenha feito uma normalização dos dados de treinamento, normalize, também, os dados de teste

- Caso tenha selecionado um subconjunto das colunas, selecione, também, esse subconjunto dos dados de teste.

- Caso tenha feito alguma transformação de variáveis categóricas em dummies, faça, também, nos dados de teste.

**Atenção:** Para os dados de teste, mantenha as mesmas colunas (e na mesma ordem) que foram usados no treinamento.

Com base em seu modelo treinado (suponha que ele se chame `model`), verifique a acurácia obtida.

In [ ]:
y_pred_test = model.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_test)
print(accuracy)

# Em caso de empate, avalie a equidade em relação ao componente racial!

# **Equidade em Aprendizado de Máquina**

Além de procurar por melhores desempenhos em modelos de aprendizado de máquina, há também preocupações éticas de seu uso. São frequentes os casos nos quais há disparidades éticas associadas a aspectos sensíveis da população, como gênero, raça, faixa etária, etc.

Com o intuito de eliminar tais disparidades, diversas regulações sobre o uso da Inteligência Artificial vem surgindo pelo mundo todo. Via de regra, a recomendação (ou, mais precisamente, a *obrigação*) é que os modelos de intelgência artificial / aprendizado de máquina sejam capazes de lidar com os problemas práticos mas que não gerem impactos negativos (dentre eles, os ligados às disparidades éticas) na população.

Para ilustrar esses cenários "injustos", vamos avaliar e comparar os desempenhos do classificador, nos dados COMPAS, para brancos e negros.

Nos comandos abaixo, há um exemplo com os dados de treinamento. No caso do modelo que foi desenvolvido para o projeto, avalie os desempenhos nos dados de teste.

In [ ]:
import pandas as pd

dados = pd.read_csv('https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Datasets/data_compas_projC1_train.csv')
dados = dados.dropna()
dados

In [ ]:
dados = pd.get_dummies(dados, columns=['sex', 'age_cat', 'race', 'c_charge_degree'])
dados

In [ ]:
X = dados.drop(columns=["score_risk"])
y = dados['score_risk']

In [ ]:
from sklearn.model_selection import train_test_split

seed = 20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=seed)

from sklearn.tree import DecisionTreeClassifier

seed = 20
modelo = DecisionTreeClassifier(criterion='entropy',random_state=seed)
modelo.fit(X_train, y_train)

In [ ]:
# Avaliando o modelo - Emprega o conjunto de teste

y_pred = modelo.predict(X_test)

In [ ]:
# Medidas de desempenho - Acurácia

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

In [ ]:
X_test

In [ ]:
# Comparando a acurácia em relação aos brancos e negros

accuracy_brancos = accuracy_score(y_test[X_test['race_Caucasian'] == 1], y_pred[X_test['race_Caucasian'] == 1])
print("Acurácia para brancos:", accuracy_brancos)

accuracy_negros = accuracy_score(y_test[X_test['race_African-American'] == 1], y_pred[X_test['race_African-American'] == 1])
print("Acurácia para negros:", accuracy_negros)

In [ ]:
# Comparando o número de indivíduos erroneamente classificados como reincidentes, em relação aos brancos e negros

erro_brancos = len(y_test[(X_test['race_Caucasian'] == 1) & (y_test[y_test == -1]) & (y_test[y_pred == 1])])/len(y_test[X_test['race_Caucasian'] == 1])
print("Erro para brancos:", erro_brancos)

erro_negros = len(y_test[(X_test['race_African-American'] == 1) & (y_test[y_test == -1]) & (y_test[y_pred == 1])])/len(y_test[X_test['race_African-American'] == 1])
print("Erro para negros:", erro_negros)

print('Diferença:', erro_negros - erro_brancos)

In [ ]:
# Comparando o número de indivíduos erroneamente classificados como não-reincidentes, em relação aos brancos e negros

erro_brancos = len(y_test[(X_test['race_Caucasian'] == 1) & (y_test[y_test == 1]) & (y_test[y_pred == -1])])/len(y_test[X_test['race_Caucasian'] == 1])
print("Erro para brancos:", erro_brancos)

erro_negros = len(y_test[(X_test['race_African-American'] == 1) & (y_test[y_test == 1]) & (y_test[y_pred == -1])])/len(y_test[X_test['race_African-American'] == 1])
print("Erro para negros:", erro_negros)

print('Diferença:', erro_brancos - erro_negros)

# Agora, testem no modelo de vocês o quanto ele está criando vieses raciais!